In [4]:
import pyreadr
import numpy as np
import xarray as xr
from bcolz import carray #Can also use ctable
import pandas as pd
import shutil
import opendatasets as ods
import datashader as ds
import dask

# Download the data from Kaggle

You'll need to sign up with Kaggle, then go to the account page and create an API token before running this.

In [2]:
ods.download("https://www.kaggle.com/averkij/tennessee-eastman-process-simulation-dataset")

Skipping, found downloaded files in ".\tennessee-eastman-process-simulation-dataset" (use force=True to force download)


## File structure

This dataset consists of four files:
* TEP_FaultFree_Testing.RData (47.3 MB)
* TEP_FaultFree_Training.RData (24.7 MB)
* TEP_Faulty_Testing.RData (836.9 MB)
* TEP_Faulty_Training.RData (494.1 MB)

The "FaultFree" files contain simulation runs that demonstrate completely normal behaviour. The "Faulty" files contain simulations where a fault is introduced either one hour (training data) or eight hours (testing data) into the simulation. Simulations in the training files ran for 500 time steps (25 hours), while simulations in the test sets are larger (960 samples, 48 hours)

Columns 4 to 55 contain the actual measurements, while column 1 contains the fault number from 0 to 20, where 0 means no fault. To keep this simple, I'm going to convert this to 0 or 1 (no fault or a fault).

## Data structure

Column two contains `simulationRun`, a number from 1 to 500 in the training data, that determines what random seed was used to make that simulation. Importantly, multiple simulations using **the same `simulationRun` value** do exist. This happens in the "Faulty" files, where the simulation is run once for each fault. In the "FaultFree" files, there's only one simulation per `simulationRun`.

This does mean that the first hour of a simulation in the training data appears 21 times (once for the fault-free simulation and 20 times for the fault simulations). I'm going to solve this the easy way by dropping the first hour of each training simulation and eight hours of each testing simulation.

## Loading RData files

This format is used by the R community, but for our purposes we need something that (a) works in Python and (b) doesn't need to be loaded entirely into RAM.

The `pyreadr` module loads RData frames into Pandas dataframes. Unfortunately, it loads the entire dataset into RAM. Here's an example of loading an object called `fault_free_training`.

In [63]:
# r_data = pyreadr.read_r('tennessee-eastman-process-simulation-dataset/TEP_FaultFree_Training.RData', use_objects=['fault_free_training'])['fault_free_training']
r_data = pyreadr.read_r('tennessee-eastman-process-simulation-dataset/TEP_Faulty_Testing.RData', use_objects=['faulty_testing'])['faulty_testing']
r_data

,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,1,1.0,1,0.25171,3672.4,4466.3,9.5122,27.057,42.473,2705.6,...,54.494,24.527,59.710,22.357,40.149,40.074,47.955,47.300,42.100,15.345
1,1,1.0,2,0.25234,3642.2,4568.7,9.4145,26.999,42.586,2705.2,...,53.269,24.465,60.466,22.413,39.956,36.651,45.038,47.502,40.553,16.063
2,1,1.0,3,0.24840,3643.1,4507.5,9.2901,26.927,42.278,2703.5,...,54.000,24.860,60.642,22.199,40.074,41.868,44.553,47.479,41.341,20.452
3,1,1.0,4,0.25153,3628.3,4519.3,9.3347,26.999,42.330,2703.9,...,53.860,24.553,61.908,21.981,40.141,40.066,48.048,47.440,40.780,17.123
4,1,1.0,5,0.21763,3655.8,4571.0,9.3087,26.901,42.402,2707.7,...,53.307,21.775,61.891,22.412,37.696,38.295,44.678,47.530,41.089,18.681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9599995,20,500.0,956,0.26494,3719.6,4536.7,9.2265,26.875,42.401,2708.3,...,54.382,26.018,62.068,20.708,37.589,35.121,45.496,42.653,40.816,15.846
9599996,20,500.0,957,0.25252,3724.0,4494.3,9.1873,27.221,41.999,2706.0,...,54.236,25.098,61.557,20.655,40.934,38.391,43.699,42.832,41.853,18.486
9599997,20,500.0,958,0.25164,3700.8,4537.3,9.2514,26.659,42.180,2704.7,...,53.722,25.185,61.169,20.650,40.694,35.961,45.643,43.147,40.538,18.127
9599998,20,500.0,959,0.29097,3641.8,4525.2,9.3053,26.823,42.234,2705.0,...,54.185,28.771,61.140,20.323,39.774,39.807,45.989,43.318,40.826,17.305


In [62]:
del r_data

NameError: name 'r_data' is not defined

### What do we want to achieve?

A useful fault detector takes in a window of data (or one sample) and tells you whether the fault is happening at that point in time. Sometimes you can predict that a fault will happen, but that's not the focus here. The data needs to be rearranged to make it easy to see where one simulation ends and another starts. To do this, I'll reshape the data into `[num_simulations, num_samples, num_features]`. It also needs to be converted from a dataframe to an array that lives on disk. Reshaping the array will make it easier to get all the rows for individual simulations. Bcolz will provide the disk-backed array.

In [1]:
# File name, object name, and finally the starting sample of the data to be extracted.
training_data_paths = [
    ('tennessee-eastman-process-simulation-dataset/TEP_FaultFree_Training.RData', 'fault_free_training', -1),
    ('tennessee-eastman-process-simulation-dataset/TEP_Faulty_Training.RData', 'faulty_training', 20)
]

testing_data_paths = [
    ('tennessee-eastman-process-simulation-dataset/TEP_FaultFree_Testing.RData', 'fault_free_testing', -1),
    ('tennessee-eastman-process-simulation-dataset/TEP_Faulty_Testing.RData', 'faulty_testing', 8*20)
]

In [50]:
def data_to_bcolz(data, data_output_name, labels_output_name):
    '''
    Convert the dataframe into a bcolz array, splitting the features and labels into separate arrays.
    '''
    pass

def load_data(spec, data_output_name, labels_output_name):
    output = None
    labels = None
    feature_len = 52
    for file, key, fault_start in spec:
        print(f'Read {file}')
        data = pyreadr.read_r(file, use_objects=[key])[key]
        print('Convert to bcolz.ctable')
        sim_length = data['sample'].max()
        num_simulations = data.shape[0] / sim_length
        if output is None:
            shutil.rmtree(f'{data_output_name}.bcolz', ignore_errors=True)
            shutil.rmtree(f'{label_output_name}.bcolz', ignore_errors=True)
#             output = ctable.fromdataframe(data, rootdir=f'{data_output_name}.bcolz')
#             labels = ctable.fromdataframe(data, rootdir=f'{label_output_name}.bcolz')
            output = carray()
        else:
            shutil.rmtree(f'{key}_data.bcolz', ignore_errors=True)
            shutil.rmtree(f'{key}_labels.bcolz', ignore_errors=True)
            output.append(ctable.fromdataframe(data, rootdir=f'{key}_data.bcolz'))
            output.append(ctable.fromdataframe(data, rootdir=f'{key}_labels.bcolz'))
            shutil.rmtree(f'{key}_data.bcolz', ignore_errors=False)
            shutil.rmtree(f'{key}_labels.bcolz', ignore_errors=False)
        print(f'Done {file}')
        del data
    return output, labels

In [51]:
training_data = load_data(training_data_paths, 'training_data')
training_data

Read tennesee-eastman-archive/TEP_FaultFree_Training.RData
Convert to bcolz.ctable
Done tennesee-eastman-archive/TEP_FaultFree_Training.RData
Read tennesee-eastman-archive/TEP_Faulty_Training.RData
Convert to bcolz.ctable
Done tennesee-eastman-archive/TEP_Faulty_Training.RData


ctable((57750000,), [('faultNumber', '<f8'), ('simulationRun', '<f8'), ('sample', '<i4'), ('xmeas_1', '<f8'), ('xmeas_2', '<f8'), ('xmeas_3', '<f8'), ('xmeas_4', '<f8'), ('xmeas_5', '<f8'), ('xmeas_6', '<f8'), ('xmeas_7', '<f8'), ('xmeas_8', '<f8'), ('xmeas_9', '<f8'), ('xmeas_10', '<f8'), ('xmeas_11', '<f8'), ('xmeas_12', '<f8'), ('xmeas_13', '<f8'), ('xmeas_14', '<f8'), ('xmeas_15', '<f8'), ('xmeas_16', '<f8'), ('xmeas_17', '<f8'), ('xmeas_18', '<f8'), ('xmeas_19', '<f8'), ('xmeas_20', '<f8'), ('xmeas_21', '<f8'), ('xmeas_22', '<f8'), ('xmeas_23', '<f8'), ('xmeas_24', '<f8'), ('xmeas_25', '<f8'), ('xmeas_26', '<f8'), ('xmeas_27', '<f8'), ('xmeas_28', '<f8'), ('xmeas_29', '<f8'), ('xmeas_30', '<f8'), ('xmeas_31', '<f8'), ('xmeas_32', '<f8'), ('xmeas_33', '<f8'), ('xmeas_34', '<f8'), ('xmeas_35', '<f8'), ('xmeas_36', '<f8'), ('xmeas_37', '<f8'), ('xmeas_38', '<f8'), ('xmeas_39', '<f8'), ('xmeas_40', '<f8'), ('xmeas_41', '<f8'), ('xmv_1', '<f8'), ('xmv_2', '<f8'), ('xmv_3', '<f8'), ('xm

In [11]:
testing_data = load_data(testing_data_paths, 'testing_data')
testing_data

Read tennesee-eastman-archive/TEP_FaultFree_Testing.RData
Read tennesee-eastman-archive/TEP_Faulty_Testing.RData


ctable((10080000,), [('faultNumber', '<i4'), ('simulationRun', '<f8'), ('sample', '<i4'), ('xmeas_1', '<f8'), ('xmeas_2', '<f8'), ('xmeas_3', '<f8'), ('xmeas_4', '<f8'), ('xmeas_5', '<f8'), ('xmeas_6', '<f8'), ('xmeas_7', '<f8'), ('xmeas_8', '<f8'), ('xmeas_9', '<f8'), ('xmeas_10', '<f8'), ('xmeas_11', '<f8'), ('xmeas_12', '<f8'), ('xmeas_13', '<f8'), ('xmeas_14', '<f8'), ('xmeas_15', '<f8'), ('xmeas_16', '<f8'), ('xmeas_17', '<f8'), ('xmeas_18', '<f8'), ('xmeas_19', '<f8'), ('xmeas_20', '<f8'), ('xmeas_21', '<f8'), ('xmeas_22', '<f8'), ('xmeas_23', '<f8'), ('xmeas_24', '<f8'), ('xmeas_25', '<f8'), ('xmeas_26', '<f8'), ('xmeas_27', '<f8'), ('xmeas_28', '<f8'), ('xmeas_29', '<f8'), ('xmeas_30', '<f8'), ('xmeas_31', '<f8'), ('xmeas_32', '<f8'), ('xmeas_33', '<f8'), ('xmeas_34', '<f8'), ('xmeas_35', '<f8'), ('xmeas_36', '<f8'), ('xmeas_37', '<f8'), ('xmeas_38', '<f8'), ('xmeas_39', '<f8'), ('xmeas_40', '<f8'), ('xmeas_41', '<f8'), ('xmv_1', '<f8'), ('xmv_2', '<f8'), ('xmv_3', '<f8'), ('xm

In [26]:
training_data[training_data.cols.names[3:]][[10, 20, 30]]

array([(0.2348 , 3677.4, 4489.8, 9.3199, 26.695, 42.014, 2703.9, 75.193, 120.39, 0.35435, 80.241, 48.487, 2632.9, 26.304, 48.203, 3102.2, 23.319, 65.774, 230.92, 341.2 , 94.645, 77.569, 32.148, 8.9493, 26.111, 6.7796, 18.826, 1.6824, 32.876, 13.811, 23.921, 1.2803, 18.58 , 2.2525, 4.8822, 2.2569, 0.014489 , 0.81503, 0.1046  , 54.279, 44.352, 62.645, 54.542, 23.133, 61.425, 21.93 , 42.262, 33.648, 42.375, 47.328, 40.344, 17.198),
       (0.27833, 3649.7, 4479.9, 9.3486, 26.387, 42.564, 2701.5, 75.073, 120.4 , 0.33729, 80.384, 50.172, 2630.3, 27.059, 50.066, 3097.6, 22.868, 65.649, 226.86, 341.45, 94.518, 77.321, 32.294, 9.0822, 26.056, 6.9624, 18.749, 1.6767, 33.065, 13.966, 23.52 , 1.3563, 18.466, 2.2213, 4.8492, 2.232 , 0.018032 , 0.87043, 0.10962 , 53.559, 43.529, 63.137, 53.947, 27.761, 60.589, 21.743, 39.398, 38.607, 46.686, 46.688, 41.585, 18.294),
       (0.22515, 3689.6, 4525.4, 9.4095, 27.133, 42.395, 2698.9, 75.073, 120.41, 0.32419, 80.437, 50.174, 2627.2, 24.056, 51.358, 3096

In [27]:
del training_data
del testing_data

## Dask experiment

In [37]:
from dask.distributed import Client

In [72]:
dask_client = Client(n_workers=4, threads_per_worker=1, memory_limit='1GB')

H:\ProgramData\anaconda3\envs\ml\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52490 instead
  warnings.warn(


In [65]:
ddf = dask.dataframe.from_pandas(r_data, chunksize=500)

In [66]:
dask.dataframe.to_csv(ddf, 'test_data*.csv')

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ("('from_pandas-b2a754d4e979bd5e514c95463b41180e', 0)", <WorkerState 'tcp://127.0.0.1:60860', name: 1, memory: 0, processing: 19200>)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory

In [73]:
ddf = dask.dataframe.read_csv('test_data*.csv')

In [74]:
ddf.repartition(npartitions=5000)

,Unnamed: 0,Unnamed: 0.1,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,xmeas_6,xmeas_7,xmeas_8,xmeas_9,xmeas_10,xmeas_11,xmeas_12,xmeas_13,xmeas_14,xmeas_15,xmeas_16,xmeas_17,xmeas_18,xmeas_19,xmeas_20,xmeas_21,xmeas_22,xmeas_23,xmeas_24,xmeas_25,xmeas_26,xmeas_27,xmeas_28,xmeas_29,xmeas_30,xmeas_31,xmeas_32,xmeas_33,xmeas_34,xmeas_35,xmeas_36,xmeas_37,xmeas_38,xmeas_39,xmeas_40,xmeas_41,xmv_1,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
npartitions=5000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [75]:
ddf.head()

,Unnamed: 0,Unnamed: 0.1,faultNumber,simulationRun,sample,xmeas_1,xmeas_2,xmeas_3,xmeas_4,xmeas_5,...,xmv_2,xmv_3,xmv_4,xmv_5,xmv_6,xmv_7,xmv_8,xmv_9,xmv_10,xmv_11
0,0,0,0.0,1.0,1,0.25038,3674.0,4529.0,9.2320,26.889,...,53.744,24.657,62.544,22.137,39.935,42.323,47.757,47.510,41.258,18.447
1,1,1,0.0,1.0,2,0.25109,3659.4,4556.6,9.4264,26.721,...,53.414,24.588,59.259,22.084,40.176,38.554,43.692,47.427,41.359,17.194
2,2,2,0.0,1.0,3,0.25038,3660.3,4477.8,9.4426,26.875,...,54.357,24.666,61.275,22.380,40.244,38.990,46.699,47.468,41.199,20.530
3,3,3,0.0,1.0,4,0.24977,3661.3,4512.1,9.4776,26.758,...,53.946,24.725,59.856,22.277,40.257,38.072,47.541,47.658,41.643,18.089
4,4,4,0.0,1.0,5,0.29405,3679.0,4497.0,9.3381,26.889,...,53.658,28.797,60.717,21.947,39.144,41.955,47.645,47.346,41.507,18.461


In [76]:
ddf.min().compute()

Unnamed: 0          0.000000
Unnamed: 0.1        0.000000
faultNumber         0.000000
simulationRun       1.000000
sample              1.000000
xmeas_1             0.122450
xmeas_2          3511.800000
xmeas_3          4336.900000
xmeas_4             8.972700
xmeas_5            25.951000
xmeas_6            41.394000
xmeas_7          2672.300000
xmeas_8            72.649000
xmeas_9           120.310000
xmeas_10            0.285180
xmeas_11           78.980000
xmeas_12           45.874000
xmeas_13         2598.900000
xmeas_14           20.752000
xmeas_15           45.853000
xmeas_16         3072.700000
xmeas_17           20.125000
xmeas_18           63.933000
xmeas_19          187.440000
xmeas_20          334.220000
xmeas_21           93.967000
xmeas_22           76.133000
xmeas_23           30.970000
xmeas_24            8.483400
xmeas_25           24.954000
xmeas_26            6.420400
xmeas_27           17.527000
xmeas_28            1.554900
xmeas_29           31.367000
xmeas_30      

In [77]:
del ddf

In [78]:
dask_client.shutdown()

In [79]:
del dask_client

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [34]:
ddf.xmeas_1[:500]

NotImplementedError: Series getitem is only supported for other series objects with matching partition structure